In [1]:
import numpy as np
import geopandas as gpd
from scipy.spatial import KDTree
from tqdm.autonotebook import tqdm
from scipy.ndimage import gaussian_filter
from fuzzywuzzy import process
import contextily as cx

import warnings, traceback

from IndexUtils import * 
from TileUtils import *
from featurematching import *
from ReadDate import *
from concurrent.futures import ThreadPoolExecutor

# TILED INFERENCE
import sahi
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction, predict
from scipy.spatial.distance import cdist
import easyocr
from scipy.fftpack import fft2, ifft2, fftshift
from scipy.ndimage import center_of_mass

from sklearn.cluster import DBSCAN

from skimage.morphology import skeletonize

from scipy.interpolate import Rbf


%load_ext autoreload
%autoreload 2

C:\Users\fhacesga\AppData\Local\Temp\ipykernel_19464\3096312912.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


WARNING  Python>=3.10 is required, but Python==3.8.18 is currently installed 


In [2]:
referencing_dir  = r"D:\RECTDNN\processing\2024-10-11_15-08-46\referencing_3\\"

In [3]:
with open("SmartDictV2.p", 'rb') as f:
    tile_coords = pickle.load(f)

In [4]:
base_input_path = r"D:\RECTDNN\uncompress\\"
# LIST ALL IMAGES IN DIRECTORY
image_files = []
wildcards = ['480287', '48201', '480296']

# wildcards = ['480296']
for w in wildcards:
    print(f"Potential detections {w}: {len([a for a in list(tile_coords.keys()) if w in a])}")
    # print([a for a in list(tile_coords.keys()) if w in a])
    image_files.extend(glob.glob(f"{base_input_path}/*{w}*"))
image_files = [f for f in image_files if 'w' not in os.path.splitext(f)[1]]

# FILTER IMAGES USING HEURISTICS TO GET TILEINDICES
patterns = ["IND", "_1."]
index_files = [file for pattern in patterns for file in glob.glob(base_input_path + "\\*" + pattern + "*.*[!w]*")]
filtered_files = [file for file in image_files if len(os.path.basename(file)) < 12]
index_files.extend(filtered_files)

# GET ACTUAL TILES
tiles       = list(set(image_files) - set(index_files))
tiles_bns   = [os.path.basename(tile).split(".")[0] for tile in tiles]

Potential detections 480287: 99
Potential detections 48201: 295
Potential detections 480296: 51


In [ ]:
# r"D:\RECTDNN\processing\2024-10-11_15-08-46\referencing\48201C0930J.tif"
# tile_name = f"48201C0660L"
# tile_name = f"48201C0930J"
# tile_name = f"48201C0530J"
# tile_name = f"48201C0150G"
# tile_name = f"48201C0290H"
# for tile_name in ["48201C0290H", "48201C0930J", "48201C0530J", "48201C0660L"]:
# for tile_name in ["48201C0930J"]:

try:
    reader
except:
    reader = easyocr.Reader(['en']) 

def find_file_with_extension(filename, allowed_extensions, directory):
    for ext in allowed_extensions:
        file_path = os.path.join(directory, f"{filename}.{ext}")
        if os.path.isfile(file_path):
            return file_path
    return None 

counter = 0
bad_counter = 0

# tile_names = [os.path.basename(a).split(".")[0] for a in glob.glob(corner_dir + "*")][5:]
for tile_name in (pbar := tqdm(tiles_bns)): # tile_names: 

    pbar.set_description(f"Bad counter: {bad_counter}")

    counter = counter
    print(f"Processing {tile_name}" + "-" * 50)
    

    # im_file = r"D:\RECTDNN\uncompress\\" + tile_name
    im_file = find_file_with_extension(tile_name, ['tif', 'jpg', 'png'], base_input_path)

    if im_file is None:
        print(f"COULD NOT FIND {tile_name} IMAGE " + '-' * 20)
        continue

    curr_tn = os.path.basename(im_file.split(".")[0])[:-1]
    curr_tf = os.path.basename(im_file.split(".")[0])[-1]

    ra = rio.open(im_file)
    bounds  = ra.bounds
    image   = ra.read(1)

    bounds, RLNN = findBounds(im_file, None)

    if len(bounds[0]) < 1:
        print(f"COULD NOT FIND BOUNDS, USING IMAGE EXTENTS {tile_name}")
        bbox = [0, 0, image.shape[0], image.shape[1]]
        datebox = [ int(image.shape[1] * 0.75), int(image.shape[0] * 0.75), image.shape[1], image.shape[0]]
    else:
        bbox = bounds[0].boxes.xyxy.numpy().astype(np.int32)[0].flatten()
        # ASSUME DATE IS IN LOWER RIGHT OF IMAGE
        datebox = [bbox[2], int(3 * image.shape[0] // 4), image.shape[1], image.shape[0]]

    current_tile_coords = tile_coords.get(curr_tn, None)
    if current_tile_coords is None:
        print(f"Could not find tile in detections {curr_tn}")
        bad_counter = bad_counter + 1
        continue
    
    # GET DATE OF CURRENT TILE
    tile_datetime = extractDate(image[datebox[1]:datebox[3], datebox[0]:datebox[2]], reader, imsize=4092, filter_values=['revised', 'effective'])

    if tile_datetime[0] is not None: 
        # IF FOUND, FIND CLOSEST DATE WITHIN CURRENT KEY
        curr_bbox, closest_date = getBBOXClosestDate(extractDatesFromDict(tile_coords[curr_tn], curr_tf), tile_datetime[0][0])

        # IF CLOSEST DATE WITHIN CURRENT KEY IS OVER 10 YEARS, RETRY WITH ENTIRE DICT
        if abs(closest_date - tile_datetime[0][0]).days > 3650:
            curr_bbox, closest_date_gen = getBBOXClosestDate(extractDatesFromDict(tile_coords[curr_tn], 0), tile_datetime[0][0])
            print(f"CLOSEST DATE IN KEY IS TOO FAR ({abs(closest_date - tile_datetime[0][0]).days} days), using general instead ({closest_date_gen})")
    else: 
        print(f"Could not find date in {tile_datetime[1]}")
        # IF NOT FOUND, USE DEFAULT (0)
        current_tf_coords = current_tile_coords.get(curr_tf, None)
        if current_tf_coords is None:
            print(f"Could not find tile extension in detections {curr_tn} - {curr_tf}, using default {list(current_tile_coords.keys())[0]} from options {current_tile_coords.keys()}")
            curr_tf = list(current_tile_coords.keys())[0]
            curr_bbox = tile_coords[curr_tn][curr_tf]["coords"][0]

    real_x, real_y = bbox_to_coords_realworld(curr_bbox)
    pic_x, pic_y   = bbox_to_coords_raster(bbox)

    affine = affineTransformation(pic_x, pic_y, real_x, real_y)
    transform = bboxTransformToCRS(affine.matrix, image)

    ref_dir = f"{referencing_dir}{tile_name}.tif"

    with rio.open(ref_dir, 'w',
        driver='GTiff', count=1, dtype=image.dtype,
        height=image.shape[0], width=image.shape[1],
        crs=f'EPSG:3857',
        transform=rio.Affine(*transform.flatten()[:6])) as dst:
            dst.write(image, 1) 
    

  0%|          | 0/859 [00:00<?, ?it/s]

Processing 4802960160B--------------------------------------------------
Using 1979-12-11 00:00:00 for tile date 1979-12-11 00:00:00
Processing 4802870310D--------------------------------------------------
Using 1985-09-27 00:00:00 for tile date 1985-09-27 00:00:00
Processing 48201C0840K--------------------------------------------------
Could not find date in ['HARRIS  COUNTY', 'TEXAS AND', 'INCORPORATED   AREAS', 'PANEL 840 OF 1135', '{SEE MA? INDEX FOR BANELs NOT PRINTED;', 'CONTAWNS', 'QMMUNTY _NUMBER FAVEL   SUFFIX', 'Fort BEND Cou', 'DNO 2 & 481272 Co', 'HARRIS COUN', 'UNINCORPORATED ARE45 480287  0840', 'HoustoNCrcoRRt 430296   C#c', 'U', 'Map  NUMBER', '48201C0840 K', 'MAp  REVISED:', "30'", 'APRIL 20, 2000', 'Federal Emergency Management Agency', 'FOR']
Processing 480296A_53--------------------------------------------------
Could not find tile in detections 480296A_5
Processing 480287B_38--------------------------------------------------
Could not find tile in detections 480287

11/08/2024 10:04:07 - WARNING - rasterio._env -   CPLE_AppDefined in D:/RECTDNN/uncompress/48201C0970L.tif: _TIFFVSetField:D:/RECTDNN/uncompress/48201C0970L.tif: Null count for "Tag 32934" (type 4, writecount -3, passcount 1)
11/08/2024 10:04:07 - INFO - rasterio._env -   GDAL signalled an error: err_no=1, msg='_TIFFVSetField:D:/RECTDNN/uncompress/48201C0970L.tif: Null count for "Tag 32934" (type 4, writecount -3, passcount 1)'


Processing 48201C0970L--------------------------------------------------
Using 2007-06-18 00:00:00 for tile date 2007-06-18 00:00:00
Processing 480296A_21--------------------------------------------------
Could not find tile in detections 480296A_2
Processing 4802870475D--------------------------------------------------
Using 1982-03-30 00:00:00 for tile date 1982-03-30 00:00:00
Processing 48201C0215G--------------------------------------------------
Could not find key G for current tile from options [0, 'L'], using closest date from all keys
Using 1996-11-23 00:00:00 for tile date 1990-09-28 00:00:00
Processing 480287B_3--------------------------------------------------
Could not find tile in detections 480287B_
Processing 480296A_109--------------------------------------------------
Could not find tile in detections 480296A_10
Processing 480287B_35--------------------------------------------------
Could not find tile in detections 480287B_3
Processing 4802870500C---------------------

11/08/2024 10:04:48 - WARNING - rasterio._env -   CPLE_AppDefined in D:/RECTDNN/uncompress/48201C0705L.tif: _TIFFVSetField:D:/RECTDNN/uncompress/48201C0705L.tif: Null count for "Tag 32934" (type 4, writecount -3, passcount 1)
11/08/2024 10:04:48 - INFO - rasterio._env -   GDAL signalled an error: err_no=1, msg='_TIFFVSetField:D:/RECTDNN/uncompress/48201C0705L.tif: Null count for "Tag 32934" (type 4, writecount -3, passcount 1)'


...

Processing 48201C0930L--------------------------------------------------
Using 2007-06-18 00:00:00 for tile date 2007-06-18 00:00:00
Processing 48201C0645J--------------------------------------------------
Using 1996-11-23 00:00:00 for tile date 1996-11-06 00:00:00
Processing 48201C0265H--------------------------------------------------
Could not find key H for current tile from options [0, 'L', 'M', 'G'], using closest date from all keys
Using 1992-09-30 00:00:00 for tile date 1992-09-30 00:00:00
Processing 48201C0390M--------------------------------------------------
Using 2013-10-16 00:00:00 for tile date 2013-10-16 00:00:00


11/08/2024 10:24:38 - WARNING - rasterio._env -   CPLE_AppDefined in D:/RECTDNN/uncompress/48201C0920L.tif: _TIFFVSetField:D:/RECTDNN/uncompress/48201C0920L.tif: Null count for "Tag 32934" (type 4, writecount -3, passcount 1)
11/08/2024 10:24:38 - INFO - rasterio._env -   GDAL signalled an error: err_no=1, msg='_TIFFVSetField:D:/RECTDNN/uncompress/48201C0920L.tif: Null count for "Tag 32934" (type 4, writecount -3, passcount 1)'


Processing 48201C0920L--------------------------------------------------
Could not find date in ['TEXAS', 'AND INCORPORATED AREAS', 'L', 'PANEL 920 OF 1150', '(SEE   MAP   INDEX   FOR   FIRM   PANEL   LAYOUT)', 'CONTAINS _', 'COMMUNTY NUMBER > PANEL _> SUFFIX', 'W', 'DEERPAFK CITY OF 480201 0820', 'PASADEIA CHTY OF 480307 0820', 'HOUSTON, CITY OF 480206 0820', 'HL', 'WD', 'Notice to Usor: The Map Numbar shown bolow should be usod', 'whan   placing   map   ordars;   the  Community   Number   shown', 'above should be usod on insuranca applications for the subject', 'communiy', 'MAP NUMBER', '48201C0920L', 'MAP REVISED:', 'AND', 'JUNE 18 , 2007', 'Federal Emergency Management Agency', 'DEPAR', 'SECUR']
Processing 480296A_99--------------------------------------------------
Could not find tile in detections 480296A_9
Processing 480296A_26--------------------------------------------------
Could not find tile in detections 480296A_2
Processing 4802870055E-------------------------------------

11/08/2024 10:24:46 - WARNING - rasterio._env -   CPLE_AppDefined in D:/RECTDNN/uncompress/48201C1085L.tif: _TIFFVSetField:D:/RECTDNN/uncompress/48201C1085L.tif: Null count for "Tag 32934" (type 4, writecount -3, passcount 1)
11/08/2024 10:24:46 - INFO - rasterio._env -   GDAL signalled an error: err_no=1, msg='_TIFFVSetField:D:/RECTDNN/uncompress/48201C1085L.tif: Null count for "Tag 32934" (type 4, writecount -3, passcount 1)'


Using 1988-02-04 00:00:00 for tile date 1988-02-04 00:00:00
Processing 48201C1085L--------------------------------------------------


In [ ]:
ra_ref.close()